In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

In [2]:
#imports for Python and Pandas

import json
from pandas.io.json import json_normalize

In [3]:
#use normalize to create a Pandas dataframe out of the JSON file

df = pd.read_json('world_bank_projects.json') 

# Question 1 :
+ Find the 10 countries with most projects

In [4]:
#output countires with most projects, using head() to limit to top ten

df.countryshortname.value_counts().head(10)

Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Bangladesh            12
Nepal                 12
Morocco               12
Mozambique            11
Africa                11
Name: countryshortname, dtype: int64

In [5]:
#create a new dataframe that excludes Africa which is not a country.

df_countries = df[df.countryshortname != ("Africa")]

## Answer 1

In [6]:
print('Top 10 Countries with Most Projects')
df_countries.countryshortname.value_counts().head(10)

Top 10 Countries with Most Projects


China                 19
Indonesia             19
Vietnam               17
India                 16
Yemen, Republic of    13
Bangladesh            12
Nepal                 12
Morocco               12
Mozambique            11
Brazil                 9
Name: countryshortname, dtype: int64

# Question 2: 
+ Find the top 10 major project themes

In [7]:
#examine mjtheme_namecode and find that we have nested fields 
df.mjtheme_namecode.head()

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object

In [8]:
#load in the json file
data = json.load((open('world_bank_projects.json')))
# use normalize to create a dataframe of project codes and names, countries, and id's
df_themes = json_normalize(data, 'mjtheme_namecode', ['countryshortname', 'id'])
df_themes.head(10)

,code,name,countryshortname,id
0,8,Human development,Ethiopia,P129828
1,11,,Ethiopia,P129828
2,1,Economic management,Tunisia,P144674
3,6,Social protection and risk management,Tunisia,P144674
4,5,Trade and integration,Tuvalu,P145310
5,2,Public sector governance,Tuvalu,P145310
6,11,Environment and natural resources management,Tuvalu,P145310
7,6,Social protection and risk management,Tuvalu,P145310
8,7,Social dev/gender/inclusion,"Yemen, Republic of",P144665
9,7,Social dev/gender/inclusion,"Yemen, Republic of",P144665


In [9]:
len(np.unique(df_themes['id']))

500

In [10]:
df_themes.name.value_counts().head(10)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64

Here we can see that some project id's have multiple entries for same code/name

In [11]:
#remove duplicates where the same code appears more than once in any project
df_themes = df_themes.drop_duplicates(['id', 'code'], keep='first')
df_themes.head(10)

,code,name,countryshortname,id
0,8,Human development,Ethiopia,P129828
1,11,,Ethiopia,P129828
2,1,Economic management,Tunisia,P144674
3,6,Social protection and risk management,Tunisia,P144674
4,5,Trade and integration,Tuvalu,P145310
5,2,Public sector governance,Tuvalu,P145310
6,11,Environment and natural resources management,Tuvalu,P145310
7,6,Social protection and risk management,Tuvalu,P145310
8,7,Social dev/gender/inclusion,"Yemen, Republic of",P144665
10,5,Trade and integration,Lesotho,P144933


In [12]:
#output top ten themes
df_themes.name.value_counts().head(10)

Environment and natural resources management    135
Rural development                               135
Public sector governance                        126
Human development                               118
Social protection and risk management           110
                                                107
Financial and private sector development        104
Social dev/gender/inclusion                     103
Trade and integration                            56
Urban development                                37
Name: name, dtype: int64

## Answer 2

In [13]:
print('Top 10 Major Project Themes')
df_themes.name.value_counts().head(11) #accounting for missing names

Top 10 Major Project Themes


Environment and natural resources management    135
Rural development                               135
Public sector governance                        126
Human development                               118
Social protection and risk management           110
                                                107
Financial and private sector development        104
Social dev/gender/inclusion                     103
Trade and integration                            56
Urban development                                37
Economic management                              28
Name: name, dtype: int64

We can see there are entires without names. This will be addressed in the next problem.

# Question 3: 
+ Create a new dataframe with missing project names from answer 2 filled in

### Previous Answer - Used front fill on list to generate missing project names.

In [14]:
#start by setting 'code' as our index
code_df = df_themes.set_index(['code']).sort_index()
code_df.head(10)

,name,countryshortname,id
code,,,
1,Economic management,Ukraine,P131234
1,Economic management,Lao People's Democratic Republic,P129825
1,Economic management,Seychelles,P132425
1,Economic management,Armenia,P132948
1,Economic management,Sao Tome and Principe,P130925
1,Economic management,Uruguay,P131440
1,Economic management,Poland,P130459
1,Economic management,"Macedonia, former Yugoslav Republic of",P133791
1,Economic management,Pacific Islands,P133255


In [15]:
#fill in the missing names using ffill
code_df['name'] = code_df.name.replace('', method='ffill')
code_df.head()

,name,countryshortname,id
code,,,
1,Economic management,Ukraine,P131234
1,Economic management,Lao People's Democratic Republic,P129825
1,Economic management,Seychelles,P132425
1,Economic management,Armenia,P132948
1,Economic management,Sao Tome and Principe,P130925


In [16]:
#reset index
themes_df = code_df.reset_index()

In [17]:
#group by project name and sort to gove top ten
top_themes = themes_df.groupby(['name']).size().sort_values(ascending=False).head(10)

### Revised Answer - Create dataframe containg only project names and codes and merge into original dataframe

In [18]:
#create copy of df_themes to use for our revised answer
df_themes_copy = json_normalize(data, 'mjtheme_namecode', ['countryshortname', 'id'])
df_themes_copy.head()

,code,name,countryshortname,id
0,8,Human development,Ethiopia,P129828
1,11,,Ethiopia,P129828
2,1,Economic management,Tunisia,P144674
3,6,Social protection and risk management,Tunisia,P144674
4,5,Trade and integration,Tuvalu,P145310


In [19]:
#replace the missing names with Nan
df_themes_copy['name'] = df_themes_copy['name'].replace('', np.nan)

In [20]:
#create a dataframe consisting of code and name
df_themes0 = df_themes_copy[['code','name']]
df_themes0.head(10)

,code,name
0,8,Human development
1,11,NaN
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
7,6,Social protection and risk management
8,7,Social dev/gender/inclusion
9,7,Social dev/gender/inclusion


In [21]:
#drop all duplicates
df_themes01 = df_themes0.drop_duplicates()
df_themes01.head(10)

,code,name
0,8,Human development
1,11,NaN
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
8,7,Social dev/gender/inclusion
11,4,Financial and private sector development
13,6,NaN


In [22]:
#check to see if we have 1 of each project name
df_themes01.name.value_counts()

Environment and natural resources management    1
Economic management                             1
Urban development                               1
Human development                               1
Social protection and risk management           1
Rule of law                                     1
Public sector governance                        1
Trade and integration                           1
Financial and private sector development        1
Social dev/gender/inclusion                     1
Rural development                               1
Name: name, dtype: int64

In [23]:
#create a new dataframe containing each project name and the corresponding code
df_lookup = df_themes01[df_themes01['name'].notnull()]
df_lookup.head(11)

,code,name
0,8,Human development
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration
5,2,Public sector governance
6,11,Environment and natural resources management
8,7,Social dev/gender/inclusion
11,4,Financial and private sector development
18,10,Rural development
53,9,Urban development


In [24]:
#merge the original dataframe with our new dataframe, creating a new name column without missing values
df_corrected = df_themes_copy.merge(df_lookup, on='code')

df_corrected.head(10)

,code,name_x,countryshortname,id,name_y
0,8,Human development,Ethiopia,P129828,Human development
1,8,NaN,China,P127033,Human development
2,8,Human development,Madagascar,P132616,Human development
3,8,Human development,Cambodia,P146271,Human development
4,8,Human development,Cambodia,P146271,Human development
5,8,Human development,Cambodia,P146271,Human development
6,8,Human development,Tajikistan,P146109,Human development
7,8,Human development,Tajikistan,P146109,Human development
8,8,Human development,Lao People's Democratic Republic,P143025,Human development
9,8,Human development,Angola,P122700,Human development


In [25]:
#drop duplicate values for projects that have the same id and code
df_corrected = df_corrected.drop_duplicates(['id', 'code', 'name_y'], keep='first')
df_corrected.head(10)

,code,name_x,countryshortname,id,name_y
0,8,Human development,Ethiopia,P129828,Human development
1,8,NaN,China,P127033,Human development
2,8,Human development,Madagascar,P132616,Human development
3,8,Human development,Cambodia,P146271,Human development
6,8,Human development,Tajikistan,P146109,Human development
8,8,Human development,Lao People's Democratic Republic,P143025,Human development
9,8,Human development,Angola,P122700,Human development
10,8,Human development,Bangladesh,P145118,Human development
11,8,Human development,"Congo, Republic of",P128628,Human development
12,8,NaN,Sao Tome and Principe,P098847,Human development


In [26]:
#set our corrected dataframe to code and project name with out missing values, name_y
df_corrected = df_corrected[['code','name_y']]


## Answer 3

### Previous answer

In [27]:
print('Top 10 Major Project Themes: ')
top_themes

Top 10 Major Project Themes: 


name
Environment and natural resources management    157
Rural development                               148
Public sector governance                        140
Human development                               127
Financial and private sector development        119
Social protection and risk management           116
Social dev/gender/inclusion                     115
Trade and integration                            61
Urban development                                40
Economic management                              33
dtype: int64

We can see that after removing duplicate entries and filling in missing names that several of the themes have switched places from our answer in problem 2.

### Revised Answer

In [28]:
print('Top 10 Major Project Themes')
df_corrected.name_y.value_counts().head(10)

Top 10 Major Project Themes


Environment and natural resources management    157
Rural development                               148
Public sector governance                        140
Human development                               128
Financial and private sector development        119
Social protection and risk management           116
Social dev/gender/inclusion                     114
Trade and integration                            61
Urban development                                40
Economic management                              33
Name: name_y, dtype: int64

The results are indeed different. Human development increased by 1 to 128 and Social dev/gender/includion decreased by 1. ffill may have been an "easier" solution, but by merging a dataframe with the corrected pairs of codes and names, you avoid errors as the fill function moves through the data.
